In [1]:
import json
import random
import string
import nltk 
import numpy as np

#reading json file
file= open('intents.json').read()

chats = json.loads(file)

len(chats['intents'])
chats



{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What help you p

In [2]:
# identifying features and target.
# tokenizing words is achieved  using nltk.word_tokenize.
# data_X represents features and data_Y represents tags of the patterns.
#  words list contains all tokens , classes contains the corresponding tags, both are sorted und duplicate-cleaned.
#The first step here is to prepare our input data by identifying features represented by chats
# and goals represented by tags. Additionally, we need to tokenize the chat texts with nltk.word_tokenize. 
#Some data cleansing is required to remove the tags and punctuation as these cannot provide meaningful information
#to the model.
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
ignore = ['?']
words =[]
classes = []
docs = []
data_Y =[]

for chat in chats['intents']:
    for patt in chat['patterns']:
        tokens = nltk.word_tokenize(patt)#tokenize the pattern
        words.extend(tokens)##add tokens to words list
        docs.append((tokens, chat['tag'])) ## append the pattern to
        data_Y.append(chat['tag'])
        
    #add tags to classes list
        if chat['tag'] not in classes:
            classes.append(chat['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\la2022\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#lemmatize all words (get the stems) and convert them to lower cases, remove punctuations
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(docs), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

121 documents
43 classes ['HR_related_problem', 'Location', 'Weather', 'about', 'appointment status', 'cabin', 'check_leave', 'commission', 'competitors_in_market', 'configuration', 'connect_people', 'cost_lowering', 'customer_satisfaction', 'domain', 'email_id', 'factors_impacting_sale', 'forgot_password', 'gadgets', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'options', 'order_components', 'order_tracking', 'predict_delay', 'predict_performance', 'project_handling_queries', 'search_department', 'search_person_by_id', 'solve_problems', 'supplier_info', 'thanks', 'turnover', 'version_update']
226 unique stemmed words ["'s", ',', '.', '23a12', '23a31', '32712', '345a23', '431b67', '561a24', '562b78', '@', 'a', 'abc', 'abx', 'accid', 'ai', 'am', 'an', 'analys', 'and', 'anyon', 'appoint', 'appoit', 'ar', 'at', 'awesom', 'bas', 'be', 'been', 'benefit', 'bhat', 'bor', 'bye',

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\la2022\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# create training data
# bag-of-words is used to convert words into numbers. 
#  words and classes act as a vocabulary for patterns and tags respectively.
# We’ll use words and classes to create two arrays, of the same lengths as the sizes of two vocabulary lists, for every chat.
# The array will have values 1 if the word is present in the patterns/tag of the chat, otherwise 0. 

training = []
#output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in docs:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
print(len(classes))
# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])  

43


In [24]:
len(train_x[0])
len(words)

226

In [25]:
#building neural network model,we'll create Keras’ Sequential model.
#The input to this network will be the array train_X
# the model has 3 layers with the first having 128 neurons, the second having 64 neurons, and the last layer having the same number of neurons as number of classes.
# Next, to reach the correct weights, Adam optimizer is used. loss function is  categorical cross-entropy function.
# And, the metric  to evaluate the model is 'accuracy'.
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
model = Sequential()
# the number of units(neurons) could affect the resulted training accuracy
model.add(Dense(128, input_shape =(len(train_x[0]),), activation ='relu'))#input layer
model.add(Dropout(0.5))#dropout layer
model.add(Dense(64, activation ='relu'))#hidden layer
model.add(Dropout(0.5))
# output layer, classification model(activation function: softmax)
model.add(Dense(len(train_y[0]), activation ='softmax'))
adam =tf.keras.optimizers.Adam(learning_rate=0.01)    
model.compile(loss= 'categorical_crossentropy', 
              optimizer =adam,
              metrics=['accuracy'])
print(model.summary)
model.fit(x=train_x, y=train_y, epochs=1000, batch_size=128)


<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x00000242DB73BB20>>
Epoch 1/1000
1/1 [==============================] - 1s 505ms/step - loss: 3.7659 - accuracy: 0.0496
Epoch 2/1000
1/1 [==============================] - 0s 16ms/step - loss: 3.6834 - accuracy: 0.0992
Epoch 3/1000
1/1 [==============================] - 0s 18ms/step - loss: 3.6309 - accuracy: 0.0992
Epoch 4/1000
1/1 [==============================] - 0s 9ms/step - loss: 3.5270 - accuracy: 0.1488
Epoch 5/1000
1/1 [==============================] - 0s 7ms/step - loss: 3.3417 - accuracy: 0.1901
Epoch 6/1000
1/1 [==============================] - 0s 9ms/step - loss: 3.2504 - accuracy: 0.2314
Epoch 7/1000
1/1 [==============================] - 0s 16ms/step - loss: 3.1343 - accuracy: 0.2066
Epoch 8/1000
1/1 [==============================] - 0s 10ms/step - loss: 3.0527 - accuracy: 0.2727
Epoch 9/1000
1/1 [==============================] - 0s 8ms/step - loss: 2.8387 - accuracy: 0.2479
Epoch 10/1000


1/1 [==============================] - 0s 14ms/step - loss: 0.1518 - accuracy: 0.9421
Epoch 164/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.1065 - accuracy: 0.9587
Epoch 165/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.1213 - accuracy: 0.9587
Epoch 166/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.1048 - accuracy: 0.9587
Epoch 167/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.1553 - accuracy: 0.9587
Epoch 168/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.1081 - accuracy: 0.9669
Epoch 169/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.1091 - accuracy: 0.9835
Epoch 170/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.1457 - accuracy: 0.9421
Epoch 171/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.1445 - accuracy: 0.9504
Epoch 172/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.1063 - accuracy: 0.9587
Epoch 1

1/1 [==============================] - 0s 18ms/step - loss: 0.0978 - accuracy: 0.9587
Epoch 326/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0921 - accuracy: 0.9587
Epoch 327/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0844 - accuracy: 0.9669
Epoch 328/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.1465 - accuracy: 0.9587
Epoch 329/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0556 - accuracy: 0.9835
Epoch 330/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.1671 - accuracy: 0.9421
Epoch 331/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.1073 - accuracy: 0.9669
Epoch 332/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1355 - accuracy: 0.9669
Epoch 333/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.1722 - accuracy: 0.9421
Epoch 334/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.1326 - accuracy: 0.9256
Epoch 335/

1/1 [==============================] - 0s 18ms/step - loss: 0.0816 - accuracy: 0.9669
Epoch 488/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0508 - accuracy: 0.9835
Epoch 489/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.1094 - accuracy: 0.9917
Epoch 490/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.1460 - accuracy: 0.9504
Epoch 491/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.1732 - accuracy: 0.9504
Epoch 492/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.0754 - accuracy: 0.9752
Epoch 493/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0625 - accuracy: 0.9752
Epoch 494/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.2012 - accuracy: 0.9504
Epoch 495/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0764 - accuracy: 0.9835
Epoch 496/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.1579 - accuracy: 0.9587
Epoch 497/

1/1 [==============================] - 0s 16ms/step - loss: 0.0415 - accuracy: 0.9752
Epoch 650/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.0569 - accuracy: 0.9835
Epoch 651/1000
1/1 [==============================] - 0s 16ms/step - loss: 0.1449 - accuracy: 0.9587
Epoch 652/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.2563 - accuracy: 0.9091
Epoch 653/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.2300 - accuracy: 0.9587
Epoch 654/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.2021 - accuracy: 0.9339
Epoch 655/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.0614 - accuracy: 0.9752
Epoch 656/1000
1/1 [==============================] - 0s 14ms/step - loss: 0.1277 - accuracy: 0.9669
Epoch 657/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.0477 - accuracy: 0.9752
Epoch 658/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.2144 - accuracy: 0.9504
Epoch 6

1/1 [==============================] - 0s 18ms/step - loss: 0.0241 - accuracy: 1.0000
Epoch 812/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0616 - accuracy: 0.9752
Epoch 813/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.2795 - accuracy: 0.9339
Epoch 814/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1635 - accuracy: 0.9587
Epoch 815/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.0711 - accuracy: 0.9669
Epoch 816/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.1701 - accuracy: 0.9587
Epoch 817/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0869 - accuracy: 0.9669
Epoch 818/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.0675 - accuracy: 0.9669
Epoch 819/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.1615 - accuracy: 0.9587
Epoch 820/1000
1/1 [==============================] - 0s 6ms/step - loss: 0.1472 - accuracy: 0.9669
Epoch 821/1000

1/1 [==============================] - 0s 11ms/step - loss: 0.1811 - accuracy: 0.9339
Epoch 974/1000
1/1 [==============================] - 0s 744us/step - loss: 0.1300 - accuracy: 0.9421
Epoch 975/1000
1/1 [==============================] - 0s 15ms/step - loss: 0.1105 - accuracy: 0.9752
Epoch 976/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1176 - accuracy: 0.9504
Epoch 977/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.1182 - accuracy: 0.9669
Epoch 978/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.1389 - accuracy: 0.9835
Epoch 979/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.0855 - accuracy: 0.9504
Epoch 980/1000
1/1 [==============================] - 0s 18ms/step - loss: 0.0945 - accuracy: 0.9669
Epoch 981/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.1463 - accuracy: 0.9587
Epoch 982/1000
1/1 [==============================] - 0s 7ms/step - loss: 0.1928 - accuracy: 0.9339
Epoch 9

In [ ]:
# model evaluation with unknown data
# chat preparation functions
def sent_clean(sent):
    # tokenizing the pattern
    tokens = nltk.word_tokenize(sent)
    # stemming each word
    sent_words = [stemmer.stem(word.lower()) for word in tokens]
    return sent_words

# returning bag of words of the pattern
def bw(sent, words):
    # tokenizing the pattern
    sent_words = sent_clean(sent)
    # create bag of words
    bag = [0]*len(words)  
    for s in sent_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                

    return(np.array(bag))

In [ ]:
# classify the pattern with a tag, and generate the appropriate response
error_threshold = 0.30
def classify(sent):
    # predict class probabilities from the model
    #tag_prob = model.predict(np.array([bw(sent, words)]))[0]# for sequential model 
    tag_prob = model.predict([bw(sent, words)])[0] # for dnn model
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(tag_prob) if r > error_threshold]
    # sort probabilities in descending order
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))# get the tag of index r[0] und prob_value is r[1]
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for chat in chats['intents']:
                # find a tag matching the first result (highst probability)
                if chat['tag'] == results[0][0]:
                    # print out a random response of the corresponding tag
                    return print(random.choice(chat['responses']))

            results.pop(0)

In [ ]:
print('enter 0 if you dont want to chat with this Chatbot')
while True:
    
    message = input('')
    if message==0:
        break
    res = response(message)
    print(res)
 